<a href="https://colab.research.google.com/github/Mookkyduke/Project_DM/blob/main/Backtesting_%E0%B8%A5%E0%B9%88%E0%B8%B2%E0%B8%AA%E0%B8%B8%E0%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install vectorbt
!pip install pandas_ta
!pip install yfinance



 ---part 1-----




In [134]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
import vectorbt as vbt
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
import plotly.figure_factory as ff
import matplotlib as mpl
import plotly.graph_objects as go
%matplotlib inline
from plotly.subplots import make_subplots


In [135]:
symbols = ['BTC-USD']
start_date = '2015-10-1'
end_date = '2022-10-1'

In [136]:
price = vbt.YFData.download(symbols,start=start_date,end=end_date,interval='1D',missing_index='drop')
price = price.loc[(price.wrapper.index >= start_date) & (price.wrapper.index < end_date)]

In [137]:
df = pd.DataFrame()
df = price.data[symbols[0]]
df.drop(['Volume','Dividends','Stock Splits'],axis=1,inplace=True)

In [168]:
# Calculate SMA (Simple Moving Average) and EMA (Exponential Moving Average)
df['SMA_50'] = df['Close'].rolling(window=50).mean()
df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()

# Calculate RSI (Relative Strength Index)
def calculate_rsi(data, window_length):
    diff = data.diff(1)
    gain = diff.where(diff > 0, 0)
    loss = -diff.where(diff < 0, 0)
    avg_gain = gain.rolling(window=window_length).mean()
    avg_loss = loss.rolling(window=window_length).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = calculate_rsi(df['Close'], 14)

# Calculate Stochastic RSI
def calculate_stoch_rsi(rsi, window_length):
    min_rsi = rsi.rolling(window=window_length).min()
    max_rsi = rsi.rolling(window=window_length).max()
    stoch_rsi = (rsi - min_rsi) / (max_rsi - min_rsi)
    return stoch_rsi * 100

stoch_rsi_k = calculate_stoch_rsi(df['RSI'], 14)
stoch_rsi_d = stoch_rsi_k.rolling(window=3).mean()

# Create SMA, EMA, and RSI data
SMA = go.Scatter(x=df.index, y=df['SMA_50'], mode='lines', name='SMA 50', marker=dict(color='blue'), opacity=0.5)
EMA_50 = go.Scatter(x=df.index, y=df['EMA_50'], mode='lines', name='EMA 50', marker=dict(color='orange'), opacity=0.5)
EMA_200 = go.Scatter(x=df.index, y=df['EMA_200'], mode='lines', name='EMA 200', marker=dict(color='green'), opacity=0.5)

In [169]:
# Setting Layout for RSI Chart
layout = go.Layout(
    title="BTC",
    plot_bgcolor="white",
    legend=dict(
        itemclick="toggleothers",
        itemdoubleclick="toggle"),
    hovermode='x',
    hoverdistance=100,
    spikedistance=1000,
    xaxis=dict(
        title="Date",
        linecolor="black",
        showspikes=True,
        spikethickness=2,
        spikedash="dot",
        spikecolor="#999999",
        spikemode="across",
        showgrid=False
    ),
    yaxis=dict(
        domain=[0.4, 1],
        title="Price",
        linecolor="black",
        showgrid=False,
    ),
    yaxis2=dict(
        domain=[0, 0.3],
        anchor="x2",
        title="RSI",
        linecolor="black",
        showgrid=False,
    )
)

In [170]:
data = []

# Add RSI and RSI threshold lines to the chart
date_generated = pd.date_range(df.index[0], periods=len(df))
rsi = go.Scatter(x=df.index, y=df['RSI'], mode='lines', name='RSI', marker=dict(color='Red'), opacity=0.5, yaxis="y2", xaxis='x1')
rsi_l_line = go.Scatter(x=df.index, y=pd.Series(30, index=date_generated), mode='lines', name='RSI', marker=dict(color='Black'), opacity=0.5, yaxis="y2", xaxis='x1')
rsi_u_line = go.Scatter(x=df.index, y=pd.Series(70, index=date_generated), mode='lines', name='RSI', marker=dict(color='Black'), opacity=0.5, yaxis="y2", xaxis='x1')
data.extend([rsi, rsi_l_line, rsi_u_line])

# Add SMA and EMA charts to the Candlestick chart
SMA_chart = go.Scatter(x=df.index, y=df['SMA_50'], mode='lines', name='SMA 50', marker=dict(color='blue'), opacity=0.5)
EMA_50_chart = go.Scatter(x=df.index, y=df['EMA_50'], mode='lines', name='EMA 50', marker=dict(color='orange'), opacity=0.5)
EMA_200_chart = go.Scatter(x=df.index, y=df['EMA_200'], mode='lines', name='EMA 200', marker=dict(color='green'), opacity=0.5)
data.extend([SMA_chart, EMA_50_chart, EMA_200_chart])

fig = go.Figure(data=data, layout=layout)

fig.update_layout(xaxis_rangeslider_visible=False)

# Stochastic RSI Data
stoch_rsi_k_chart = go.Scatter(x=df.index, y=stoch_rsi_k, mode='lines', name='Stoch RSI %K', marker=dict(color='orange'), opacity=0.5)
stoch_rsi_d_chart = go.Scatter(x=df.index, y=stoch_rsi_d, mode='lines', name='Stoch RSI %D', marker=dict(color='purple'), opacity=0.5)

In [171]:
# Setting Layout for Stochastic RSI Chart
stoch_rsi_layout = go.Layout(
    title="Stochastic RSI",
    plot_bgcolor="white",
    hovermode='x',
    hoverdistance=100,
    spikedistance=1000,
    xaxis=dict(
        title="Date",
        linecolor="black",
        showspikes=True,
        spikethickness=2,
        spikedash="dot",
        spikecolor="#999999",
        spikemode="across",
        showgrid=False
    ),
    yaxis=dict(
        domain=[0.1, 0.3],
        title="Stoch RSI",
        linecolor="black",
        showgrid=False,
    )
)

In [172]:
# Create the Stochastic RSI Figure
stoch_rsi_fig = go.Figure(data=[stoch_rsi_k_chart, stoch_rsi_d_chart], layout=stoch_rsi_layout)

# Display both charts side by side
fig_subplots = make_subplots(rows=4, cols=1, shared_xaxes=True, row_heights=[3, 1, 1, 1])

# Add the combined Candlestick and SMA/EMA chart to the first row
fig_subplots.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], increasing_line_color='Green', decreasing_line_color='Red'), row=1, col=1)
fig_subplots.add_trace(SMA_chart, row=1, col=1)
fig_subplots.add_trace(EMA_50_chart, row=1, col=1)
fig_subplots.add_trace(EMA_200_chart, row=1, col=1)
fig_subplots.add_trace(SMA_chart, row=3, col=1)
# Add RSI chart to the second row
fig_subplots.add_trace(rsi, row=2, col=1)
fig_subplots.add_trace(rsi_l_line, row=2, col=1)
fig_subplots.add_trace(rsi_u_line, row=2, col=1)

# Add Stochastic RSI chart to the fourth row
fig_subplots.add_trace(stoch_rsi_k_chart, row=4, col=1)
fig_subplots.add_trace(stoch_rsi_d_chart, row=4, col=1)

fig_subplots.update_layout(xaxis_rangeslider_visible=False)
fig_subplots.show(config={"displayModeBar": True, "showTips": False})




---part2----




In [139]:
start_date = datetime(2015,10,1)
end_date = datetime(2022,10,1)

In [140]:
data = pd.DataFrame()
data = data.ta.ticker('BTC-USD',start=start_date,end=end_date)

In [141]:
df = data.copy()

In [142]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-10-01 00:00:00+00:00,236.003998,238.445007,235.615997,237.548996,20488800,0.0,0.0
2015-10-02 00:00:00+00:00,237.264008,238.541000,236.602997,237.292999,19677900,0.0,0.0
2015-10-03 00:00:00+00:00,237.201996,239.315002,236.944000,238.729996,16482700,0.0,0.0
2015-10-04 00:00:00+00:00,238.531006,238.968002,237.940002,238.259003,12999000,0.0,0.0
2015-10-05 00:00:00+00:00,238.147003,240.382996,237.035004,240.382996,23335900,0.0,0.0


In [143]:
df = df.drop(columns=['Dividends'])

In [144]:
df = df.drop(columns=['Stock Splits'])

In [145]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2015-10-01 00:00:00+00:00,236.003998,238.445007,235.615997,237.548996,20488800
2015-10-02 00:00:00+00:00,237.264008,238.541000,236.602997,237.292999,19677900
2015-10-03 00:00:00+00:00,237.201996,239.315002,236.944000,238.729996,16482700
2015-10-04 00:00:00+00:00,238.531006,238.968002,237.940002,238.259003,12999000
2015-10-05 00:00:00+00:00,238.147003,240.382996,237.035004,240.382996,23335900


In [146]:
df.ta.macd(append = True)
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'MACD_12_26_9',
       'MACDh_12_26_9', 'MACDs_12_26_9'],
      dtype='object')

In [147]:
df["signal"] = df.ta.ema(12,append = True) > df.ta.ema(26,append=True)
df

,Open,High,Low,Close,Volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,EMA_12,EMA_26,signal
Date,,,,,,,,,,,
2015-10-01 00:00:00+00:00,236.003998,238.445007,235.615997,237.548996,20488800,NaN,NaN,NaN,NaN,NaN,False
2015-10-02 00:00:00+00:00,237.264008,238.541000,236.602997,237.292999,19677900,NaN,NaN,NaN,NaN,NaN,False
2015-10-03 00:00:00+00:00,237.201996,239.315002,236.944000,238.729996,16482700,NaN,NaN,NaN,NaN,NaN,False
2015-10-04 00:00:00+00:00,238.531006,238.968002,237.940002,238.259003,12999000,NaN,NaN,NaN,NaN,NaN,False
2015-10-05 00:00:00+00:00,238.147003,240.382996,237.035004,240.382996,23335900,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26 00:00:00+00:00,18803.900391,19274.873047,18721.285156,19222.671875,44148798321,-515.041105,-30.525217,-484.515888,19371.987311,19887.028416,False
2022-09-27 00:00:00+00:00,19221.839844,20338.455078,18915.667969,19110.546875,58571439619,-497.745559,-10.583737,-487.161823,19331.765706,19829.511265,False
2022-09-28 00:00:00+00:00,19104.621094,19688.343750,18553.296875,19426.720703,53071298734,-453.300760,27.088850,-480.389610,19346.374167,19799.674927,False


In [148]:
df.shape

(2557, 11)

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2557 entries, 2015-10-01 00:00:00+00:00 to 2022-09-30 00:00:00+00:00
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open           2557 non-null   float64
 1   High           2557 non-null   float64
 2   Low            2557 non-null   float64
 3   Close          2557 non-null   float64
 4   Volume         2557 non-null   int64  
 5   MACD_12_26_9   2532 non-null   float64
 6   MACDh_12_26_9  2524 non-null   float64
 7   MACDs_12_26_9  2524 non-null   float64
 8   EMA_12         2546 non-null   float64
 9   EMA_26         2532 non-null   float64
 10  signal         2557 non-null   bool   
dtypes: bool(1), float64(9), int64(1)
memory usage: 286.8 KB


In [150]:
df.isnull().any()

Open             False
High             False
Low              False
Close            False
Volume           False
MACD_12_26_9      True
MACDh_12_26_9     True
MACDs_12_26_9     True
EMA_12            True
EMA_26            True
signal           False
dtype: bool

In [151]:
df.isnull().sum()

Open              0
High              0
Low               0
Close             0
Volume            0
MACD_12_26_9     25
MACDh_12_26_9    33
MACDs_12_26_9    33
EMA_12           11
EMA_26           25
signal            0
dtype: int64

In [152]:
df.isnull().mean()*100

Open             0.000000
High             0.000000
Low              0.000000
Close            0.000000
Volume           0.000000
MACD_12_26_9     0.977708
MACDh_12_26_9    1.290575
MACDs_12_26_9    1.290575
EMA_12           0.430192
EMA_26           0.977708
signal           0.000000
dtype: float64

In [153]:
df.isnull().sum()

Open              0
High              0
Low               0
Close             0
Volume            0
MACD_12_26_9     25
MACDh_12_26_9    33
MACDs_12_26_9    33
EMA_12           11
EMA_26           25
signal            0
dtype: int64

In [154]:
df.dropna(subset=['EMA_12','EMA_26','MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9'], how='any', inplace=True)
df

,Open,High,Low,Close,Volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,EMA_12,EMA_26,signal
Date,,,,,,,,,,,
2015-11-03 00:00:00+00:00,361.872986,417.899994,357.647003,403.416992,206162000,30.877748,10.069555,20.808193,323.583656,292.705908,True
2015-11-04 00:00:00+00:00,403.664001,495.562012,380.548004,411.562988,263900000,35.608802,11.840487,23.768315,337.118938,301.510136,True
2015-11-05 00:00:00+00:00,408.076996,447.561005,374.580994,386.354004,151824992,36.898697,10.504305,26.394391,344.693564,307.794867,True
2015-11-06 00:00:00+00:00,388.046997,395.835999,354.024994,374.470001,122687000,36.540788,8.117117,28.423671,349.274554,312.733766,True
2015-11-07 00:00:00+00:00,374.269012,390.585999,372.433014,386.481995,56625100,36.802178,6.702806,30.099372,354.998776,318.196598,True
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26 00:00:00+00:00,18803.900391,19274.873047,18721.285156,19222.671875,44148798321,-515.041105,-30.525217,-484.515888,19371.987311,19887.028416,False
2022-09-27 00:00:00+00:00,19221.839844,20338.455078,18915.667969,19110.546875,58571439619,-497.745559,-10.583737,-487.161823,19331.765706,19829.511265,False
2022-09-28 00:00:00+00:00,19104.621094,19688.343750,18553.296875,19426.720703,53071298734,-453.300760,27.088850,-480.389610,19346.374167,19799.674927,False


In [162]:
df.isnull().sum()

Open             0
High             0
Low              0
Close            0
Volume           0
MACD_12_26_9     0
MACDh_12_26_9    0
MACDs_12_26_9    0
EMA_12           0
EMA_26           0
signal           0
TS_Trends        0
TS_Trades        0
TS_Entries       0
TS_Exits         0
dtype: int64

In [155]:
df.shape

(2524, 11)

In [156]:
df.describe()

,Open,High,Low,Close,Volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,EMA_12,EMA_26
count,2524.000000,2524.000000,2524.000000,2524.000000,2.524000e+03,2524.000000,2524.000000,2524.000000,2524.000000,2524.000000
mean,14720.475449,15100.143645,14295.945284,14726.223852,1.852400e+10,54.862577,-0.734666,55.597243,14684.647089,14629.784512
std,16854.888050,17288.335309,16351.449711,16848.844815,2.030320e+10,1055.855229,290.022850,1004.635764,16792.169682,16716.370029
min,314.079010,323.058014,300.997009,311.084015,2.343940e+07,-5053.274080,-1694.518206,-4438.999056,323.583656,292.705908
25%,2796.007446,2895.492493,2689.532593,2805.397583,1.379762e+09,-161.206553,-49.989002,-151.207320,2658.427812,2553.730894
50%,8147.711182,8283.717773,7923.050293,8157.460205,1.379464e+10,9.738260,0.937202,8.549242,8136.934602,8069.442167
75%,19722.262695,20144.687500,19273.825684,19719.054199,3.058670e+10,267.211071,61.596048,261.722685,20301.984045,20681.423893
max,67549.734375,68789.625000,66382.062500,67566.828125,3.509679e+11,5273.808690,1305.313389,4517.102703,64182.504548,62406.255181


----part3-----

In [157]:
#Create boolean Signals(Ts_Entries,TS_Exits) for vectorbt
signal_vectorbt = df.ta.tsignals(df.signal,asbool =True,append = True)
signal_vectorbt

,TS_Trends,TS_Trades,TS_Entries,TS_Exits
Date,,,,
2015-11-03 00:00:00+00:00,True,0,False,False
2015-11-04 00:00:00+00:00,True,0,False,False
2015-11-05 00:00:00+00:00,True,0,False,False
2015-11-06 00:00:00+00:00,True,0,False,False
2015-11-07 00:00:00+00:00,True,0,False,False
...,...,...,...,...
2022-09-26 00:00:00+00:00,False,0,False,False
2022-09-27 00:00:00+00:00,False,0,False,False
2022-09-28 00:00:00+00:00,False,0,False,False


In [158]:
signal_vectorbt.loc[signal_vectorbt.TS_Entries==True]

,TS_Trends,TS_Trades,TS_Entries,TS_Exits
Date,,,,
2016-02-16 00:00:00+00:00,True,1,True,False
2016-03-23 00:00:00+00:00,True,1,True,False
2016-05-26 00:00:00+00:00,True,1,True,False
2016-09-06 00:00:00+00:00,True,1,True,False
2017-01-22 00:00:00+00:00,True,1,True,False
2017-04-06 00:00:00+00:00,True,1,True,False
2017-07-22 00:00:00+00:00,True,1,True,False
2017-09-30 00:00:00+00:00,True,1,True,False
2018-02-21 00:00:00+00:00,True,1,True,False


In [163]:
port = vbt.Portfolio.from_signals(df.Close,
                                entries=signal_vectorbt.TS_Entries,
                                exits=signal_vectorbt.TS_Exits,
                                freq='D',
                                init_cash=100000,
                                fees=0.0025,
                                slippage=0.0025
)

In [164]:
port.plot().show()

In [165]:
port.stats()

Start                          2015-11-03 00:00:00+00:00
End                            2022-09-30 00:00:00+00:00
Period                                2524 days 00:00:00
Start Value                                     100000.0
End Value                                 7995802.026597
Total Return [%]                             7895.802027
Benchmark Return [%]                         4716.799847
Max Gross Exposure [%]                             100.0
Total Fees Paid                            573647.796061
Max Drawdown [%]                               62.445098
Max Drawdown Duration                  551 days 00:00:00
Total Trades                                          31
Total Closed Trades                                   31
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                    45.16129
Best Trade [%]                                364.077735
Worst Trade [%]                